In [1]:
import overpy
api = overpy.Overpass()

# fetch all ways and nodes
result = api.query("""
    way(42.2872773, -71.274711, 42.300573, -71.2510069) ["highway"];
    (._;>;);
    out body;
    """)

# Gets all of the valid nodes/points that are on roads or paths.
all_nodes = {}
for way in result.ways:
    for node in way.nodes:
        all_nodes[node] = all_nodes.get(node, 0) + 1

# Creates a list of the points that are intersections.
intersections = []
for node in all_nodes:
    if all_nodes[node] > 1:
        intersections.append(node)

In [2]:
import folium
from IPython.display import HTML, display

vertices = []
for vertex in intersections:
    vertices.append([vertex.lat, vertex.lon])
    
olin_coord = [42.294574, -71.261107]
my_map = folium.Map(location=olin_coord, zoom_start=13, tiles='OpenStreetMap', width=640, height=480)
# my_map._build_map()
# mapWidth, mapHeight = 640, 480
# srcdoc = my_map.HTML.replace('""', '&quot;')
# embed = HTML('<iframe srcdoc="{}" '
#              'style="width: {}px; height: {}px; display:block; width: 50%; margin: 0 auto; '
#              'border: none"></iframe>'.format(srcdoc, width, height))
[folium.CircleMarker(vertices[i], radius=1, color='#0080bb', fill_color='#0080bb').add_to(my_map) for i in range(len(vertices))]
my_map

In [3]:
class Edge():
    ''' Defines edges, which are the parts of roads or paths that connect the nodes (intersections) to one another. An edge contains a start node, an end node, a list of points, and a length. '''

    def __init__(self):
        ''' Initializes the edge with an empty list of points and a length of zero '''
        self.node_list = []
        self.length = 0

    def set_start_node(self, start_node):
        ''' Defines the first end node of an edge '''
        self.start = start_node

    def set_end_node(self, end_node):
        ''' Defines the second end node of an edge '''
        self.end = end_node

    def add_node(self, node):
        ''' Adds a point to the list of points that comprise the edge '''
        self.node_list.append(node)

    def add_multiple_nodes(self, nodes):
        ''' Adds a list of points to the list of points that comprise the edge '''
        self.node_list.extend(nodes)

    def update_distance(self):
        ''' Updates the length of the edge '''
        if len(self.node_list) > 0:
            first_coord = (self.start.lat, self.start.lon)
            last_coord = (self.end.lat, self.end.lon)
            self.length += vincenty((self.node_list[0].lat, self.node_list[0].lon), first_coord).km
            self.length += vincenty((self.node_list[-1].lat, self.node_list[-1].lon), last_coord).km
            for i in range(len(self.node_list) - 1):
                self.length += vincenty((self.node_list[i+1].lat, self.node_list[i+1].lon), (self.node_list[i].lat, self.node_list[i].lon)).km

    def get_bearing(self):
        bearing = Geodesic.WGS84.Inverse(self.start.lat, self.start.lon, self.end.lat, self.end.lon)['azi1']
        if bearing < 0.0:
            return bearing + 360.0
        else:
            return bearing


def find_nodes_and_edges():
    ''' Taking all of the paths/roads in the given area, this breaks up those paths/roads into nodes and edges, which it stores in a dictionary and list, respectively. '''
    edge_list = []
    for way in result.ways:
        #if way.id == 570834600:
        #print(way.nodes)
        breakpoints = []
        for i in range(len(way.nodes)):
            if way.nodes[i] in intersections:
                # if i != 0 and i != (len(way.nodes) - 1):
                breakpoints.append(i)
        if breakpoints == []:
            new_edge = Edge()
            new_edge.set_start_node(way.nodes[0])
            new_edge.set_end_node(way.nodes[-1])
            if len(way.nodes) > 2:
                new_edge.add_multiple_nodes(way.nodes[1:-1])
            #print(new_edge.node_list)
            new_edge.update_distance()
            edge_list.append(new_edge)
        else:
            new_ways = []
            for x in range(len(breakpoints) - 1):
                new_ways.append(way.nodes[breakpoints[x]:breakpoints[x+1] + 1])
            #print(new_ways)
            for y in new_ways:
                new_edge = Edge()
                new_edge.set_start_node(y[0])
                new_edge.set_end_node(y[-1])
                if len(y) > 2:
                    new_edge.add_multiple_nodes(y[1:-1])
                #print(new_edge.node_list)
                new_edge.update_distance()
                edge_list.append(new_edge)
    return edge_list


In [4]:
import geopy
from geopy.distance import VincentyDistance, vincenty
import geographiclib
from geographiclib.geodesic import Geodesic
edge_list = find_nodes_and_edges()

In [5]:
new_vertices = []
edge_pts = []
for edge in edge_list:
    new_vertices.append([edge.start.lat, edge.start.lon])
    new_vertices.append([edge.end.lat, edge.end.lon])
    for node in edge.node_list:
        edge_pts.append([node.lat, node.lon])

In [7]:
olin_coord = [42.294574, -71.261107]
my_map = folium.Map(location=olin_coord, zoom_start=13, tiles='OpenStreetMap', width=640, height=480)
# my_map._build_map()
# mapWidth, mapHeight = 640, 480
# srcdoc = my_map.HTML.replace('""', '&quot;')
# embed = HTML('<iframe srcdoc="{}" '
#              'style="width: {}px; height: {}px; display:block; width: 50%; margin: 0 auto; '
#              'border: none"></iframe>'.format(srcdoc, width, height))
[folium.CircleMarker(new_vertices[i], radius=2, color='#0080bb', fill_color='#0080bb').add_to(my_map) for i in range(len(new_vertices))]
[folium.CircleMarker(edge_pts[i], radius=1, color='#bb0000', fill_color='#bb0000').add_to(my_map) for i in range(len(edge_pts))]
#folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(my_map)
my_map

In [8]:
# new_vertices = []
# edge_pts = []
all_points = []
# for edge in edge_list:
#     new_vertices.append([edge.start.lat, edge.start.lon])
#     new_vertices.append([edge.end.lat, edge.end.lon])
#     for node in edge.node_list:
#         edge_pts.append([node.lat, node.lon])

for i in range(len(edge_list)):
    all_points.append([])
    all_points[i].append([edge_list[i].start.lat, edge_list[i].start.lon])
    all_points[i].append([edge_list[i].end.lat, edge_list[i].end.lon])
    for node in edge_list[i].node_list:
        all_points[i].append([node.lat, node.lon])

In [55]:
olin_coord = [42.294574, -71.261107]
my_map = folium.Map(location=olin_coord, zoom_start=13, tiles='OpenStreetMap', width=640, height=480)
# my_map._build_map()
# mapWidth, mapHeight = 640, 480
# srcdoc = my_map.HTML.replace('""', '&quot;')
# embed = HTML('<iframe srcdoc="{}" '
#              'style="width: {}px; height: {}px; display:block; width: 50%; margin: 0 auto; '
#              'border: none"></iframe>'.format(srcdoc, width, height))
[folium.CircleMarker(new_vertices[i], radius=2, color='#0080bb', fill_color='#0080bb').add_to(my_map) for i in range(len(new_vertices))]
# [folium.CircleMarker(edge_pts[i], radius=1, color='#bb0000', fill_color='#bb0000').add_to(my_map) for i in range(len(edge_pts))]
[folium.PolyLine(all_points[i], color="#bb0000", weight=5, opacity=1).add_to(my_map) for i in range(len(all_points))]
# poly = folium.PolyLine(all_points[1], color="#bb0000", weight=50, opacity=1)
for i in range(len(all_points)):
    for j in range(len(all_points[i])):
        folium.CircleMarker(all_points[i][j], radius=2, color='#bb0000', fill_color='#bb0000').add_to(my_map)
    folium.PolyLine(all_points[i], color="#bb0000", weight=50, opacity=1).add_to(my_map)
    
my_map

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [41]:
all_points[1]

[[Decimal('42.2876500'), Decimal('-71.2606700')],
 [Decimal('42.2880300'), Decimal('-71.2604000')],
 [Decimal('42.2878200'), Decimal('-71.2605200')]]

In [42]:
new_vertices

[[Decimal('42.2870000'), Decimal('-71.2618500')],
 [Decimal('42.2876500'), Decimal('-71.2606700')],
 [Decimal('42.2876500'), Decimal('-71.2606700')],
 [Decimal('42.2880300'), Decimal('-71.2604000')],
 [Decimal('42.2935909'), Decimal('-71.2563410')],
 [Decimal('42.2918490'), Decimal('-71.2580790')],
 [Decimal('42.2918490'), Decimal('-71.2580790')],
 [Decimal('42.2887980'), Decimal('-71.2562820')],
 [Decimal('42.2887980'), Decimal('-71.2562820')],
 [Decimal('42.2880940'), Decimal('-71.2554080')],
 [Decimal('42.2836200'), Decimal('-71.2528300')],
 [Decimal('42.2873400'), Decimal('-71.2515200')],
 [Decimal('42.2873400'), Decimal('-71.2515200')],
 [Decimal('42.2878170'), Decimal('-71.2510270')],
 [Decimal('42.2878170'), Decimal('-71.2510270')],
 [Decimal('42.2879600'), Decimal('-71.2509510')],
 [Decimal('42.2879600'), Decimal('-71.2509510')],
 [Decimal('42.2908600'), Decimal('-71.2504200')],
 [Decimal('42.2908600'), Decimal('-71.2504200')],
 [Decimal('42.2909900'), Decimal('-71.2503960')],


In [9]:
nodes_by_id = []
id_to_node = {}
for way in result.ways:
    for node in way.nodes:
        nodes_by_id.append(node.id)
        id_to_node[node.id] = node

In [10]:
import networkx as nx
G = nx.Graph()
for vertex in intersections:
    G.add_node(vertex.id, coord=(vertex.lat, vertex.lon))
for path in edge_list:
    G.add_edge(path.start.id, path.end.id, weight=path.length)
#print(G.__getitem__(530968968))
cycle = nx.find_cycle(G, 530968968)
cycle_coords = []
for i in range(len(cycle)):
    cycle_coords.append([id_to_node[cycle[i][0]].lat, id_to_node[cycle[i][0]].lon])
    if cycle[i] == cycle[-1]:
        cycle_coords.append([id_to_node[cycle[i][1]].lat, id_to_node[cycle[i][1]].lon])
#print(cycle_coords)
olin_coord = [42.294574, -71.261107]
my_map = folium.Map(location=olin_coord, zoom_start=16, tiles='OpenStreetMap', width=640, height=480)
[folium.CircleMarker(cycle_coords[i], radius=1, color='#0080bb', fill_color='#0080bb').add_to(my_map) for i in range(len(cycle_coords))]
my_map

In [11]:
cycle = nx.find_cycle(G)
cycle_coords = []
for i in range(len(cycle)):
    cycle_coords.append([id_to_node[cycle[i][0]].lat, id_to_node[cycle[i][0]].lon])
    if cycle[i] == cycle[-1]:
        cycle_coords.append([id_to_node[cycle[i][1]].lat, id_to_node[cycle[i][1]].lon])
#print(cycle_coords)
olin_coord = [42.294574, -71.261107]
my_map = folium.Map(location=olin_coord, zoom_start=16, tiles='OpenStreetMap', width=640, height=480)
[folium.CircleMarker(cycle_coords[i], radius=1, color='#0080bb', fill_color='#0080bb').add_to(my_map) for i in range(len(cycle_coords))]
my_map

In [12]:
cycle = nx.find_cycle(G, 531000239)
cycle_coords = []
for i in range(len(cycle)):
    cycle_coords.append([id_to_node[cycle[i][0]].lat, id_to_node[cycle[i][0]].lon])
    if cycle[i] == cycle[-1]:
        cycle_coords.append([id_to_node[cycle[i][1]].lat, id_to_node[cycle[i][1]].lon])
#print(cycle_coords)
olin_coord = [42.294574, -71.261107]
my_map = folium.Map(location=olin_coord, zoom_start=16, tiles='OpenStreetMap', width=640, height=480)
[folium.CircleMarker(cycle_coords[i], radius=1, color='#0080bb', fill_color='#0080bb').add_to(my_map) for i in range(len(cycle_coords))]
my_map

In [13]:
id_to_node[530968968]

<overpy.Node id=530968968 lat=42.2929279 lon=-71.2630597>